# PARTE 1: Algoritmo Naïve Bayes

Nesta primeira parte do Trabalho você irá aplicar o algoritmo de Naïve Bayes na base de dados de risco de crédito discutida em aula. Para isso você deve primeiramente importar as bibliotecas necessárias.

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [31]:
# importe a base de dados de risco de crédito e nomeie com: dataset_risco_credito_risco_credito
dataset_risco_credito_risco_credito = pd.read_csv('dataset_risco_credito.csv')

In [46]:
# imprima a base de dados para uma primeira avaliação dos dados
# Mapear os valores da coluna 'renda' para rótulos legíveis (ex: 'Renda de 0 a 15')
renda_map = {
    '0_15': 'Renda de 0 a 15',
    '15_35': 'Renda de 15 a 35',
    'acima_35': 'Renda acima de 35'
}
# usar a variável de dataset existente (tenta a variável com nome longo e, se não existir, usa a outra)
try:
    ds = dataset_risco_credito_risco_credito
except NameError:
    ds = dataset_risco_credito

# criar coluna com rótulos legíveis; se algum valor não tiver mapeamento, mantém o original
ds['renda_label'] = ds['renda'].map(renda_map).fillna(ds['renda'])

# Plot usando a nova coluna 'renda_label' para mostrar textos amigáveis no eixo x/legenda
fig = px.bar(
    ds,
    x='renda_label',            # eixo x com rótulos legíveis
    color='risco',              # cor por classe de risco
    barmode='group',            # barras lado a lado
    color_discrete_map={'alto':'#EF553B', 'moderado':'#FFA15A', 'baixo':'#00CC96'}, # cores intuitivas
    labels={'renda_label':'Faixa de Renda', 'count':'Quantidade de Registros', 'risco':'Risco de Crédito'},
    title='Distribuição de Risco de Crédito por Faixa de Renda'
)

# Estilo do layout
fig.update_layout(
    title_font_size=22,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=16,
    plot_bgcolor='#F9F9F9'
)

fig.show(renderer="browser")

In [ ]:
# --- Alternativas de visualização (use a que preferir) ---
# Reutiliza o dataframe 'ds' criado anteriormente (com coluna 'renda_label')
# Se 'ds' não existir, tenta as outras variáveis do notebook
try:
    ds
except NameError:
    try:
        ds = dataset_risco_credito_risco_credito
    except NameError:
        ds = dataset_risco_credito

# mapa de cores consistente
color_map = {'alto':'#EF553B', 'moderado':'#FFA15A', 'baixo':'#00CC96'}

# 1) Barras empilhadas (absoluto)
fig_stack = px.histogram(ds, x='renda_label', color='risco', barmode='stack',
                      color_discrete_map=color_map,
                      labels={'renda_label':'Faixa de Renda','count':'Quantidade'},
                      title='Risco por Faixa de Renda (empilhado)')
fig_stack.update_layout(title_font_size=20)
fig_stack.show(renderer='browser')

# 2) Barras empilhadas em percentagem (útil para comparar proporções)
fig_stack_pct = px.histogram(ds, x='renda_label', color='risco', barmode='stack', barnorm='percent',
                            color_discrete_map=color_map,
                            labels={'renda_label':'Faixa de Renda','percent':'Percentual'},
                            title='Risco por Faixa de Renda (percentual)')
fig_stack_pct.update_layout(title_font_size=20)
fig_stack_pct.show(renderer='browser')

# 3) Pizza (distribuição total de risco) — também dá para filtrar por faixa de renda
fig_pie = px.pie(ds, names='risco', color='risco', color_discrete_map=color_map,
                 title='Distribuição total de Risco (todas as faixas)')
fig_pie.update_traces(textposition='inside', textinfo='percent+label')
fig_pie.show(renderer='browser')

# 4) Sunburst (hierárquico: renda -> risco)
fig_sun = px.sunburst(ds, path=['renda_label','risco'], color='risco',
                   color_discrete_map=color_map,
                   title='Risco por Faixa de Renda (sunburst)')
fig_sun.show(renderer='browser')

# 5) Heatmap de contagens (risco x renda)
counts = ds.groupby(['risco','renda_label']).size().reset_index(name='count')
pivot = counts.pivot(index='risco', columns='renda_label', values='count').fillna(0)
fig_heat = px.imshow(pivot, labels={'x':'Faixa de Renda','y':'Risco','color':'Contagem'},
                   x=pivot.columns, y=pivot.index, color_continuous_scale='Blues',
                   title='Heatmap: número de registros por Risco e Faixa de Renda')
fig_heat.update_layout(title_font_size=20)
fig_heat.show(renderer='browser')

# Dica: se quiser apenas um desses gráficos por vez, comente os fig.show() que não quiser executar.


# 1 - Pré-Processamento dos dados

a) DIVISÃO DA BASE DE DADOS

Separe a base de dados dataset_risco_credito_risco_credito em:
 - variável x, com nome: X_risco_credito
 - classe y, com nome: y_risco_credito

DICA: você pode utilizar .iloc para selecionar as colunas da matriz e .values para converter para o numpy array.

In [33]:


X_risco_credito = dataset_risco_credito.iloc[:, 0:-1].values   # todas as colunas, exceto a última
y_risco_credito = dataset_risco_credito.iloc[:, -1].values     # apenas a última coluna


print("X_risco_credito:\n", X_risco_credito[:5])  # mostra as 5 primeiras linhas
print("\ny_risco_credito:\n", y_risco_credito[:5])

X_risco_credito:
 [['ruim' 'alta' 'nenhuma' '0_15']
 ['desconhecida' 'alta' 'nenhuma' '15_35']
 ['desconhecida' 'baixa' 'nenhuma' '15_35']
 ['desconhecida' 'baixa' 'nenhuma' 'acima_35']
 ['desconhecida' 'baixa' 'nenhuma' 'acima_35']]

y_risco_credito:
 ['alto' 'alto' 'moderado' 'alto' 'baixo']


b) APLICAR LABEL ENCODER

Perceba que seus dados possuem atributos categóricos (string). Porém, para aplicar esses dados em um algoritmo de aprendizado você precisa transformá-lo em atributo numérico. 

Como você pode resolver isso?

DICA: Veja o que é e como aplicar o Label Enconder em: https://youtu.be/nLKEkBAbpQo 

In [34]:
#Transforma todos os parametros de String em valores númericos
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

for i in range(X_risco_credito.shape[1]):
    X_risco_credito[:, i] = labelencoder.fit_transform(X_risco_credito[:, i])

y_risco_credito = labelencoder.fit_transform(y_risco_credito)

print("X_risco_credito (numérico):\n", X_risco_credito[:5])
print("\ny_risco_credito (numérico):\n", y_risco_credito[:5])

X_risco_credito (numérico):
 [[2 0 1 0]
 [1 0 1 1]
 [1 1 1 1]
 [1 1 1 2]
 [1 1 1 2]]

y_risco_credito (numérico):
 [0 0 2 0 1]


c) SALVAR O ARQUIVO PRÉ-PROCESSADO

In [35]:
# como salvar o arquivo:
import pickle
with open('risco_credito.pkl', 'wb') as f:
  pickle.dump([X_risco_credito, y_risco_credito], f)

# 2 - Algoritmo Naïve Bayes

In [7]:
# importar da biblioteca sklearn o pacote Nayve Bayes
# utilizamos a distribuição estatística Gaussiana (classe GaussianNB) ou distribuição normal pois é mais usado para problemas genéricos
from sklearn.naive_bayes import GaussianNB

In [8]:
# Criar o objeto Nayve Bayes
naiveb_risco_credito = GaussianNB()

a) TREINAR O ALGORITMO

Para treinar o algoritmo, você deve gerar a tabela de probabilidades. Para isso, você pode utilizar **.fit** para gerar a tabela.

DICA: O 1º parametro são os atributos/características (x) e o 2º parametro é a classe (y).

OBS: Não se preocupe, o algoritmo faz a correção laplaciana automaticamente :) .

In [36]:
# Treinar o modelo Naive Bayes usando os dados pré-processados
naiveb_risco_credito.fit(X_risco_credito, y_risco_credito)

,priors,None
,var_smoothing,1e-09


b) FAZER A PREVISÃO

Utilize **.predict** para fazer a previsão realizada no exemplo em sala.

i) história boa, dívida alta, garantia nenhuma, renda > 35

ii) história ruim, dívida alta, garantia adequada, renda < 15

Verifique nos slides se seu resultado está correto!

In [37]:

# utilize o atributo .classes_ para mostrar as classes utilizadas pelo algoritmo
print("Classes originais:", labelencoder.inverse_transform(naiveb_risco_credito.classes_))
print("Classes utilizadas pelo algoritmo:", naiveb_risco_credito.classes_)


Classes originais: ['alto' 'baixo' 'moderado']
Classes utilizadas pelo algoritmo: [0 1 2]


In [38]:
# utilize .class_count_ para contar quantos registros tem em cada classe 
print("Número de registros em cada classe:", naiveb_risco_credito.class_count_)


entrada1 = [[2, 0, 0, 2]]  # boa, alta, nenhuma, >35
entrada2 = [[0, 0, 1, 0]]  # ruim, alta, adequada, <15

previsao1 = naiveb_risco_credito.predict(entrada1)
previsao2 = naiveb_risco_credito.predict(entrada2)

print("\nRisco da Previsão 1:", labelencoder.inverse_transform(previsao1))
print("\nRisco Previsão 2 ", labelencoder.inverse_transform(previsao2))

Número de registros em cada classe: [6. 5. 3.]

Risco da Previsão 1: ['baixo']

Risco Previsão 2  ['alto']
